### **Structured Extraction of Key Information for Trading Command Execution**

In [ ]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.5/361.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
from pprint import pprint

In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "set_chain",
      "description": "Set chain to trade on",
      "parameters": {
        "type": "object",
        "properties": {
          "chain": {
            "type": "string",
            "description": "Chain to trade on",
          },
        },
        "required": ["chain"],
      },
    }
  },
  {
    "type": "function",
    "function": {
      "name": "set_leverage",
      "description": "Adjust leverage for all available assets",
      "parameters": {
        "type": "object",
        "properties": {
          "leverage": {
            "type": "integer",
            "description": "A number between 1 and 100 specifying the leverage",
          },
        },
        "required": ["leverage"],
      },
    }
  },
  {
    "type": "function",
    "function": {
      "name": "set_default_asset",
      "description": "Set default asset to trade",
      "parameters": {
        "type": "object",
        "properties": {
          "asset": {
            "type": "string"
          },
        },
        "required": ["asset"],
      },
    }
  },
    {
        "type": "function",
        "function": {
          "name": "set_default_size",
          "description": "Adjust leverage for all available assets",
          "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "number"
          }
            },
            "required": ["size"],
          },
        }
    },
  {
        "type": "function",
        "function": {
          "name": "set_default_margin",
          "description": "Set default trading margin",
          "parameters": {
        "type": "object",
        "properties": {
          "margin": {
            "type": "number"
          }
        },
        "required": ["margin"],
          },
        }
    },
    {
        "type": "function",
        "function": {
          "name": "take_profit",
          "description": "Set a take profit order for a specified asset",
          "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "integer",
            "description": "A number between 0 and 100 denoting the percentage of the order size",
          },
          "asset": {
            "type": "string",
            "description": "The asset to set the take profit order for",
          },
          "tp": {
            "type": "integer",
            "description": "A number between 0 and 100 denoting the take profit percentage/gain in asset before take profit order is triggered",
          },
        },
        "required": ["size", "asset", "tp"],
          },
        }
    },
  {
        "type": "function",
        "function": {
          "name": "stop_loss",
          "description": "Set a stop loss order for a specified asset",
          "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "integer",
            "description": "A number between 0 and 100 denoting the percentage of the order size",
          },
          "asset": {
            "type": "string",
            "description": "The asset to set the take profit order for",
          },
          "sl": {
            "type": "integer",
            "description": "A number between 0 and 100 denoting the stop loss percentage/loss in asset before stop loss order is triggered",
          },
        },
        "required": ["size", "asset", "sl"],
          },
        }
    },
  {
        "type": "function",
        "function": {
          "name": "buy",
          "description": "Place a buy order for a specified asset",
          "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "integer",
            "description": "Order size. Default from config if not specified",
          },
          "asset": {
            "type": "string",
            "description": "Asset to buy. Default from config if not specified",
          },
          "price": {
            "type": "number",
            "description": "Buy price. Market price if unspecified",
          },
          "sl": {
            "type": "number",
            "description": "Stop loss price",
          },
          "tp": {
            "type": "number",
            "description": "Take profit price",
          },
          "chase": {
            "type": "boolean",
            "description": "Whether to chase the limit order until filled",
          },
          "dp": {
            "type": "number",
            "description": "If the chase parameter is true, discard price to stop chasing",
          },
          "tsl": {
            "type": "number",
            "description": "Trailing stop loss. If not set, the trailing stop loss will not be set. ",
          }
        },
        "required": [],
          },
        }
    },
  {
        "type": "function",
        "function": {
          "name": "twap_buy",
          "description": "Divides the total order size by the number of intervals. After the time between intervals, each piece of the divided order will be bought at market price",
          "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "number",
            "description": "Total order size",
          },
          "asset": {
            "type": "string",
            "description": "Asset to buy",
          },
          "interval": {
              "type": "integer",
              "description": "Time between intervals in minutes"
          },
          "n_interval": {
              "type": "integer",
              "description": "Number of intervals"
          },
        },
        "required": ["size", "asset", "interval_min", "n_interval"],
          },
        }
    },
  {
        "type": "function",
        "function": {
          "name": "twap_sell",
          "description": "Divides the total order size by the number of intervals. After the time between intervals, each piece of the divided order will be sold at market price",
          "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "number",
            "description": "Total order size",
          },
          "asset": {
            "type": "string",
            "description": "Asset to sell",
          },
          "interval": {
              "type": "integer",
              "description": "Time between intervals in minutes"
          },
          "n_interval": {
              "type": "integer",
              "description": "Number of intervals"
          },
        },
        "required": ["size", "asset", "interval_min", "n_interval"],
          },
        }
    },
  {
        "type": "function",
        "function": {
          "name": "pair_buy",
          "description": "Long one asset while shorting another asset in a pair",
          "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "number",
            "description": "Is the order size to split between the long and short",
          },
          "pair": {
            "type": "string",
            "description": "Is the pair to long and short. In the format of AssetToLong/AssetToShort",
          },
          "price": {
            "type": "number",
            "description": "Limit price ratio for entering the trade",
          },
          "sl": {
              "type": "number",
              "description": "Stop loss ratio"
          },
          "tp": {
              "type": "number",
              "description": "Take profit ratio"
          },
        },
        "required": ["size", "pair"],
          },
        }
    },
  {
        "type": "function",
        "function": {
          "name": "pair_sell",
          "description": "Short one asset while longing another asset in a pair",
          "parameters": {
        "type": "object",
        "properties": {
          "size": {
            "type": "number",
            "description": "Is the order size to split between the short and long",
          },
          "pair": {
            "type": "string",
            "description": "Is the pair to long and short. In the format of AssetToShort/AssetToLong",
          },
          "price": {
            "type": "number",
            "description": "Limit price ratio for entering the trade",
          },
          "sl": {
              "type": "number",
              "description": "Stop loss ratio"
          },
          "tp": {
              "type": "number",
              "description": "Take profit ratio"
          },
        },
        "required": ["size", "pair"],
          },
        }
    },
  {
        "type": "function",
        "function": {
          "name": "scale_buy",
          "description": "Scale into a long position by placing limit orders at intervals",
          "parameters": {
        "type": "object",
        "properties": {
          "total_size": {
            "type": "integer",
            "description": "The size over all intervals",
          },
          "n_intervals": {
            "type": "integer",
            "description": "Number of intervals",
          },
          "asset": {
            "type": "string",
            "description": "The asset to scale buy",
          },
          "lower": {
              "type": "number",
              "description": "The lower price to place orders"
          },
          "upper": {
              "type": "number",
              "description": "The upper price to place orders"
          },
        },
        "required": ["total_size", "n_intervals", "asset", "lower", "upper"],
          },
        }
    },
  {
        "type": "function",
        "function": {
          "name": "scale_sell",
          "description": "Scale into a short position by placing limit orders at intervals",
          "parameters": {
        "type": "object",
        "properties": {
          "total_size": {
            "type": "integer",
            "description": "The size over all intervals",
          },
          "n_intervals": {
            "type": "integer",
            "description": "Number of intervals",
          },
          "asset": {
            "type": "string",
            "description": "The asset to scale sell",
          },
          "lower": {
              "type": "number",
              "description": "The lower price to place orders"
          },
          "upper": {
              "type": "number",
              "description": "The upper price to place orders"
          },
        },
        "required": ["total_size", "n_intervals", "asset", "lower", "upper"],
          },
        }
    },
  {
        "type": "function",
        "function": {
          "name": "view_pnl",
          "description": "View the current unrealized profit and loss (PnL)",
          "parameters": {}
        }
    },
  {
        "type": "function",
        "function": {
          "name": "view_balance",
          "description": "View current wallet balance",
          "parameters": {}
        }
    },
  {
        "type": "function",
        "function": {
          "name": "view_unfilled",
          "description": "View current unfilled orders",
          "parameters": {}
        }
    },
  {
        "type": "function",
        "function": {
          "name": "view_open",
          "description": "View current open positions",
          "parameters": {}
        }
    },
  {
    "type": "function",
    "function": {
        "name": "reduce_position",
        "description": "Reduce an open position by a specified percentage",
        "parameters": {
            "type": "object",
            "properties": {
                "asset": {
                    "type": "string",
                    "description": "The asset to reduce the position for",
                },
                "percentage": {
                    "type": "number",
                    "description": "The percentage to reduce the position by",
                }
            },
            "required": ["asset", "percentage"],
        },
    }
}

]

#### **Different methods for extracting all the relevant information needed to execute the corresponding trading command:**

In [ ]:
from openai import OpenAI
from pprint import pprint

client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "If the price of bitcoin is less than ethereum, buy SOL and sell btc. independent of the price of eth and btc buy 3 DAI. if ethereum is less than bitcoin, short ethereum and long btc"


final_msg_content = f"""
List all the relevant informations required for executing the following command without further explanation or numbers

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
    ]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

pprint(completion.choices[0].message.content)

('- Current price of Bitcoin (BTC)\n'
 '- Current price of Ethereum (ETH)\n'
 '- Current price of Solana (SOL)\n'
 '- Current price of DAI\n'
 '- Account balance for BTC, ETH, SOL, and DAI\n'
 '- Trading platform or exchange API access\n'
 '- Permissions for buying, selling, shorting, and longing assets\n'
 '- Transaction fees for each trade\n'
 '- Risk management parameters (e.g., stop-loss, take-profit levels)\n'
 '- Market conditions and liquidity for BTC, ETH, SOL, and DAI\n'
 '- Order types (market, limit, etc.)\n'
 '- Position size and leverage (if applicable)\n'
 '- Confirmation of trade execution and order status')


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "If the price of bitcoin is less than ethereum, buy SOL and sell btc. independent of the price of eth and btc buy 3 DAI. if ethereum is less than bitcoin, short ethereum and long btc"

final_msg_content = f"""
List all the relevant informations required for executing the following command without further explanation or numbers

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')

print(formatted_output)

- Current price of Bitcoin (BTC)
- Current price of Ethereum (ETH)
- Current price of Solana (SOL)
- Current price of DAI
- Account balance for BTC, ETH, SOL, and DAI
- Trading platform or exchange API access
- API keys and authentication details
- Order types (market, limit, etc.)
- Position sizes for SOL, BTC, ETH, and DAI
- Leverage settings for shorting and longing
- Risk management parameters (stop-loss, take-profit)
- Transaction fees and slippage considerations
- Market conditions and liquidity
- Time of execution and order validity


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "If the price of bitcoin is less than ethereum, buy SOL and sell btc. independent of the price of eth and btc buy 3 DAI. if ethereum is less than bitcoin, short ethereum and long btc"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')

print(formatted_output)

1. Current price of Bitcoin (BTC)
2. Current price of Ethereum (ETH)
3. Current price of Solana (SOL)
4. Current price of DAI
5. Trading platform or exchange API access
6. API keys and authentication details
7. Account balance and available funds
8. Trading pair details (BTC/USD, ETH/USD, SOL/USD, DAI/USD)
9. Order types (market, limit, etc.)
10. Position sizes for SOL, BTC, ETH, and DAI
11. Risk management parameters (stop-loss, take-profit levels)
12. Transaction fees and slippage considerations
13. Market conditions and liquidity
14. Execution logic for conditional statements
15. Error handling and logging mechanisms


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "If the price of bitcoin is less than ethereum, buy SOL and sell btc. independent of the price of eth and btc buy 3 DAI. if ethereum is less than bitcoin, short ethereum and long btc"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with coding expertise. Provide a complete response, avoiding any further explanations or numerical details"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')
print(formatted_output)

1. **Market Data:**
   - Current price of Bitcoin (BTC)
   - Current price of Ethereum (ETH)
   - Current price of Solana (SOL)
   - Current price of DAI

2. **Trading Platform:**
   - Access to a trading platform or exchange that supports BTC, ETH, SOL, and DAI trading
   - API keys or credentials for the trading platform

3. **Account Information:**
   - Sufficient balance in the account to execute the trades
   - Margin account enabled for shorting ETH and longing BTC

4. **Order Types:**
   - Market order for buying SOL
   - Market order for selling BTC
   - Market order for buying DAI
   - Market order for shorting ETH
   - Market order for longing BTC

5. **Risk Management:**
   - Predefined risk tolerance levels
   - Stop-loss and take-profit levels for each trade

6. **Trading Logic:**
   - Conditional checks to compare BTC and ETH prices
   - Execution logic for buying SOL and selling BTC if BTC < ETH
   - Execution logic for buying 3 DAI regardless of price
   - Execution log

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "If the price of bitcoin is less than ethereum, buy SOL and sell btc. independent of the price of eth and btc buy 3 DAI. if ethereum is less than bitcoin, short ethereum and long btc"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with coding expertise. Provide a perfect complete response, avoiding any further explanations or numerical details"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')
print(formatted_output)

To execute the command, you will need the following information:

1. **Current Prices:**
   - Bitcoin (BTC) price
   - Ethereum (ETH) price
   - Solana (SOL) price
   - DAI price

2. **Trading Actions:**
   - Buy SOL
   - Sell BTC
   - Buy DAI
   - Short ETH
   - Long BTC

3. **Quantities:**
   - Amount of SOL to buy
   - Amount of BTC to sell
   - Amount of DAI to buy (3 DAI)
   - Amount of ETH to short
   - Amount of BTC to long

4. **Trading Platform:**
   - API access to the trading platform
   - API keys and authentication details

5. **Account Information:**
   - Available balance in the trading account
   - Margin requirements for shorting and longing

6. **Order Types:**
   - Market order or limit order for each trade

7. **Risk Management:**
   - Stop-loss and take-profit levels for each trade
   - Position sizing rules

8. **Execution Logic:**
   - Conditional checks for price comparisons
   - Sequential execution of trades based on conditions

9. **Error Handling:**
   - Pro

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "If the price of bitcoin is less than ethereum, buy SOL and sell btc. independent of the price of eth and btc buy 3 DAI. if ethereum is less than bitcoin, short ethereum and long btc"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with coding expertise. Provide a complete and detailed response, avoiding any further explanations or numerical details"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')
print(formatted_output)

To execute the command, you will need the following relevant information:

1. **Current Prices:**
   - Bitcoin (BTC) price
   - Ethereum (ETH) price
   - Solana (SOL) price
   - DAI price

2. **Trading Actions:**
   - Buy SOL
   - Sell BTC
   - Buy DAI
   - Short ETH
   - Long BTC

3. **Quantities:**
   - Amount of SOL to buy
   - Amount of BTC to sell
   - Quantity of DAI to buy (3 DAI)
   - Amount of ETH to short
   - Amount of BTC to long

4. **Account Information:**
   - Available balance in the trading account
   - Margin requirements for shorting ETH
   - Margin requirements for longing BTC

5. **Trading Platform:**
   - API access to the trading platform
   - API keys and authentication details
   - Endpoint URLs for placing buy, sell, short, and long orders

6. **Order Types:**
   - Market order
   - Limit order
   - Stop order

7. **Risk Management:**
   - Stop-loss levels
   - Take-profit levels
   - Position sizing rules

8. **Execution Logic:**
   - Conditional checks for B

### **Testing on different examples:**

#### first example:

In [ ]:
from openai import OpenAI
from pprint import pprint

client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "Sell off all assets when my master account equity goes below 1k"

final_msg_content = f"""
List all the relevant informations required for executing the following command without further explanation or numbers

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')

print(formatted_output)

- Master account identification
- Current equity of the master account
- Threshold value (1k)
- List of all assets in the master account
- Market conditions for selling assets
- Trading platform API access
- Authentication credentials for the trading platform
- Sell order execution logic
- Error handling mechanisms
- Logging and monitoring setup


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "Sell off all assets when my master account equity goes below 1k"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')

print(formatted_output)

1. Master account credentials
2. API access and keys
3. Current equity value
4. Threshold value (1k)
5. List of all assets in the master account
6. Market conditions and liquidity
7. Sell order execution method (market or limit)
8. Trading platform or broker details
9. Error handling and logging mechanisms
10. Confirmation and notification settings
11. Compliance with trading regulations and policies
12. Risk management protocols
13. Time and date of execution
14. Network and server reliability
15. Backup and recovery plans


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "Sell off all assets when my master account equity goes below 1k"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with coding expertise. Provide a perfect response, avoiding any further explanations or numerical details"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')
print(formatted_output)

1. Master account credentials
2. API access and keys
3. Current equity value of the master account
4. List of all assets in the master account
5. Market conditions and liquidity
6. Trading platform or broker details
7. Sell order execution method (market order, limit order, etc.)
8. Error handling and logging mechanisms
9. Compliance with trading regulations and broker policies
10. Notification system for successful execution or failure


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "Sell off all assets when my master account equity goes below 1k"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with coding expertise. Provide a complete response, avoiding any further explanations or numerical details"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')
print(formatted_output)

1. **Brokerage Platform**: Identify the brokerage platform you are using (e.g., Interactive Brokers, TD Ameritrade, etc.).
2. **API Access**: Ensure you have API access to your brokerage account.
3. **API Key**: Obtain your API key and secret for authentication.
4. **Account ID**: Know your master account ID.
5. **Equity Check Endpoint**: Identify the API endpoint to check account equity.
6. **Sell Order Endpoint**: Identify the API endpoint to place sell orders.
7. **Authentication Method**: Understand the authentication method required (e.g., OAuth, API key).
8. **Programming Language**: Choose a programming language (e.g., Python, JavaScript).
9. **Libraries/SDKs**: Identify any libraries or SDKs needed for API interaction.
10. **Error Handling**: Plan for error handling and retries.
11. **Rate Limits**: Be aware of API rate limits.
12. **Asset List**: Retrieve the list of all assets in the account.
13. **Order Types**: Decide on the type of sell orders (e.g., market, limit).
14. **

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "Sell off all assets when my master account equity goes below 1k"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with coding expertise. Provide a complete and detailed response, avoiding any explanations or numerical details"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')
print(formatted_output)

1. **Brokerage Platform**: Identify the brokerage platform or trading platform being used (e.g., MetaTrader, Interactive Brokers, etc.).

2. **API Access**: Ensure you have API access to the brokerage platform for automated trading.

3. **API Documentation**: Obtain the API documentation for the brokerage platform to understand the endpoints and methods available.

4. **Account Credentials**: Secure your master account credentials (API key, secret key, account ID, etc.).

5. **Programming Language**: Choose a programming language compatible with the brokerage platform's API (e.g., Python, JavaScript, etc.).

6. **Equity Check Endpoint**: Identify the API endpoint or method to check the current equity of the master account.

7. **Sell Order Endpoint**: Identify the API endpoint or method to place sell orders for all assets.

8. **Asset List Endpoint**: Identify the API endpoint or method to retrieve a list of all assets held in the master account.

9. **Threshold Value**: Define the equ

#### Second example:

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "Buy BTC when BTC funding rate is 0.01%"

final_msg_content = f"""
List all the relevant informations required for executing the following command without further explanation or numbers

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')

print(formatted_output)

- Current BTC funding rate
- BTC market price
- Available account balance
- Trading platform or exchange
- Order type (market or limit)
- Order size or quantity
- Leverage (if applicable)
- Trading fees
- Risk management parameters (e.g., stop-loss, take-profit)
- API access (if using automated trading)
- Network latency and execution speed
- Regulatory compliance and KYC status
- Market conditions and liquidity
- Historical funding rate data (for context)


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "Buy BTC when BTC funding rate is 0.01%"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')

print(formatted_output)

1. Exchange platform
2. API key and secret
3. BTC/USDT (or relevant pair) ticker symbol
4. Current BTC funding rate
5. Desired funding rate threshold (0.01%)
6. Order type (market or limit)
7. Order size (amount of BTC to buy)
8. Account balance
9. Trading fees
10. Risk management parameters (stop-loss, take-profit)
11. Network latency and execution speed
12. Error handling and retry logic
13. Logging and monitoring setup
14. Compliance with exchange's rate limits and trading rules


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "Buy BTC when BTC funding rate is 0.01%"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with coding expertise. Provide a perfect response, avoiding any explanations or numerical details"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')
print(formatted_output)

1. Trading Platform API Key
2. Trading Platform API Secret
3. Trading Platform Endpoint URL
4. BTC Funding Rate Data Source
5. BTC Funding Rate Data Endpoint
6. Current BTC Funding Rate
7. Trading Pair (BTC/USD or BTC/USDT)
8. Order Type (Market Order)
9. Order Quantity
10. Account Balance
11. Risk Management Parameters
12. Error Handling Mechanism
13. Logging Mechanism
14. Execution Time
15. Network Connectivity
16. Security Protocols


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "Buy BTC when BTC funding rate is 0.01%"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanations or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with coding expertise. Provide a complete response, avoiding any numerical details"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')
print(formatted_output)

To execute the command "Buy BTC when BTC funding rate is 0.01%", you will need the following relevant information:

1. **Exchange Platform**:
   - Name of the cryptocurrency exchange where the trade will be executed.
   - API access credentials (API key and secret).

2. **Account Information**:
   - Account balance and available funds.
   - Verification of account status (e.g., KYC completion).

3. **Market Data**:
   - Current BTC funding rate.
   - Real-time BTC price.
   - Order book depth and liquidity.

4. **Trading Parameters**:
   - Order type (e.g., market order, limit order).
   - Quantity of BTC to buy.
   - Slippage tolerance.

5. **Risk Management**:
   - Stop-loss level.
   - Take-profit level.
   - Position sizing rules.

6. **Technical Indicators** (if applicable):
   - Any additional indicators or signals that might influence the decision.

7. **Regulatory Compliance**:
   - Adherence to local and international trading regulations.
   - Tax implications of the trade.

8

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "Buy BTC when BTC funding rate is 0.01%"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanations or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with coding expertise. Provide a complete and detailed response, avoiding any further explanations or numerical details"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')
print(formatted_output)

1. **Exchange Information:**
   - Name of the exchange (e.g., Binance, BitMEX, Bybit)
   - API key and secret for authentication
   - API endpoint for placing orders
   - API endpoint for retrieving funding rate

2. **Account Information:**
   - Account balance
   - Available margin
   - Position size limits
   - Leverage settings

3. **Market Information:**
   - Current BTC price
   - Order book depth
   - Trading volume
   - Market liquidity

4. **Funding Rate Information:**
   - Current BTC funding rate
   - Historical funding rates
   - Funding rate calculation method
   - Funding rate update frequency

5. **Order Parameters:**
   - Order type (e.g., market order, limit order)
   - Order size (amount of BTC to buy)
   - Order price (if using limit order)
   - Time-in-force (e.g., GTC, IOC, FOK)

6. **Risk Management:**
   - Stop-loss level
   - Take-profit level
   - Position sizing rules
   - Risk-to-reward ratio

7. **Technical Indicators:**
   - Moving averages
   - RSI (Relativ

#### Third example:

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "sell 30 btc when btc spread is more than 0.04%"

final_msg_content = f"""
List all the relevant informations required for executing the following command without further explanation or numbers

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')

print(formatted_output)

- Current BTC price
- BTC spread percentage
- Quantity to sell (30 BTC)
- Market conditions
- Trading platform or exchange
- Account balance
- Order type (market or limit)
- Fees and commissions
- Risk management parameters
- Execution strategy
- Timing and duration of the order
- Regulatory compliance
- Confirmation and logging of the transaction


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "sell 30 btc when btc spread is more than 0.04%"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')

print(formatted_output)

1. Trading platform or exchange
2. API key and secret
3. Current BTC price
4. BTC spread calculation method
5. Account balance
6. Order type (market or limit)
7. Order size (30 BTC)
8. Spread threshold (0.04%)
9. Trading pair (BTC/USD, BTC/EUR, etc.)
10. Network fees
11. Slippage tolerance
12. Execution time
13. Error handling
14. Logging and monitoring
15. Compliance with regulations
16. Risk management strategy


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "sell 30 btc when btc spread is more than 0.04%"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanations or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with coding expertise. Provide a complete and detailed response, avoiding any further explanations or numerical details"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')
print(formatted_output)

1. **Asset to Trade**: Bitcoin (BTC)
2. **Action**: Sell
3. **Quantity**: 30 BTC
4. **Condition**: BTC spread is more than 0.04%
5. **Spread Definition**: Difference between the highest bid price and the lowest ask price
6. **Market Data Source**: Reliable cryptocurrency exchange or market data provider
7. **API Access**: Credentials and permissions to access market data and execute trades
8. **Trading Platform**: Exchange or broker that supports BTC trading
9. **Order Type**: Market order or limit order (depending on strategy)
10. **Risk Management**: Stop-loss, take-profit levels, and position sizing
11. **Compliance**: Adherence to regulatory requirements and exchange rules
12. **Network Latency**: Consideration of execution speed and data retrieval
13. **Error Handling**: Mechanisms to handle failed orders or data retrieval issues
14. **Logging**: Record of executed trades and conditions met
15. **Security**: Secure storage of API keys and sensitive information
16. **Time Frame**: 

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "sell 30 btc when btc spread is more than 0.04%"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanations or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with coding expertise. Provide a complete and detailed response, avoiding any further numerical details"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')
print(formatted_output)

To execute the command "sell 30 BTC when BTC spread is more than 0.04%," you will need the following relevant information:

1. **Trading Platform Information:**
   - Name of the trading platform or exchange.
   - API access credentials (API key and secret).

2. **Account Information:**
   - Account ID or username.
   - Available balance of BTC in the account.

3. **Market Data:**
   - Current bid price of BTC.
   - Current ask price of BTC.
   - Calculation of the spread (difference between ask and bid prices).

4. **Order Details:**
   - Order type (market order or limit order).
   - Quantity to sell (30 BTC).
   - Price condition (spread more than 0.04%).

5. **Risk Management:**
   - Stop-loss level (if applicable).
   - Take-profit level (if applicable).

6. **Execution Conditions:**
   - Real-time monitoring of the BTC spread.
   - Trigger mechanism to execute the sell order when the spread condition is met.

7. **Compliance and Security:**
   - Ensure compliance with the exchange

#### Fourth example:

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "close my btc order when spread goes less than 0.01%"

final_msg_content = f"""
List all the relevant informations required for executing the following command without further explanation or numbers

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')

print(formatted_output)

- Order type (BTC)
- Current spread value
- Spread threshold (0.01%)
- Order status (open/close)
- Trading platform or exchange
- API access or trading interface
- Authentication credentials
- Order ID or reference
- Market data feed
- Execution conditions (spread comparison)


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "close my btc order when spread goes less than 0.01%"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')

print(formatted_output)

1. Trading platform or exchange
2. API key and secret
3. Order ID or reference
4. Current BTC order details
5. Spread calculation method
6. Threshold value (0.01%)
7. Market data source
8. Authentication method
9. Error handling procedures
10. Network connectivity
11. Time synchronization
12. Logging mechanism
13. Security protocols
14. Rate limits
15. Programming language
16. Relevant libraries or SDKs
17. Execution environment
18. Notification system
19. Backup plan
20. Compliance requirements


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "close my btc order when spread goes less than 0.01%"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with coding expertise. Provide a complete and detailed response, avoiding any further numerical details"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')
print(formatted_output)

To execute the command "close my BTC order when spread goes less than 0.01%", you will need the following relevant information:

1. **Trading Platform/Exchange**:
   - Name of the trading platform or exchange where the BTC order is placed.

2. **API Access**:
   - API key and secret for authentication.
   - API endpoint for order management.

3. **Order Details**:
   - Order ID or unique identifier of the BTC order to be closed.
   - Type of order (e.g., limit, market).

4. **Spread Calculation**:
   - Current bid price of BTC.
   - Current ask price of BTC.
   - Formula to calculate the spread.

5. **Threshold Condition**:
   - Spread threshold value (0.01%).

6. **Monitoring Mechanism**:
   - Frequency of checking the spread.
   - Method to continuously monitor the spread (e.g., polling, WebSocket).

7. **Closing Mechanism**:
   - API endpoint for closing the order.
   - Parameters required to close the order (e.g., order ID, type).

8. **Error Handling**:
   - Procedures for handlin

### 5th example:

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "change all my take profits to +5% from my entry price"

final_msg_content = f"""
List all the relevant informations required for executing the following command without further explanation or numbers

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')

print(formatted_output)

- Current entry price of each position
- Current take profit levels
- Position identifiers (e.g., trade IDs)
- Account details (e.g., account ID)
- Trading platform or broker API access
- Authentication credentials for the trading platform or broker
- Market conditions and liquidity
- Risk management parameters
- Time of execution
- Asset or instrument being traded


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "change all my take profits to +5% from my entry price"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')

print(formatted_output)

1. Trading platform or broker API details
2. Authentication credentials (API key, secret)
3. Account ID or user identification
4. List of open positions or trades
5. Entry price for each position
6. Current take profit levels
7. Calculation method for +5% from entry price
8. API endpoint for modifying take profit levels
9. Required parameters for the API request
10. Error handling procedures
11. Confirmation of successful update


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "change all my take profits to +5% from my entry price"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with coding expertise. Provide a complete and detailed response, avoiding any further numerical details"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')
print(formatted_output)

To execute the command "change all my take profits to +5% from my entry price," you will need the following relevant information:

1. **Account Information:**
   - Account ID
   - API Key (if using an automated trading platform)

2. **Current Open Positions:**
   - List of all open positions
   - Entry price for each position
   - Current take profit levels for each position

3. **Trading Platform Details:**
   - Name of the trading platform or broker
   - Access to the platform's API or trading interface

4. **Market Information:**
   - Current market prices for the assets in your open positions

5. **Calculation Method:**
   - Formula to calculate 5% increase from the entry price

6. **Order Modification Details:**
   - API endpoint or interface command to modify take profit levels
   - Required parameters for the API call or command (e.g., position ID, new take profit level)

7. **Error Handling:**
   - Procedures for handling potential errors or exceptions during the modification p

#### 6th example:

In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "reverse the direction on all of my orders"

final_msg_content = f"""
List all the relevant informations required for executing the following command without further explanation or numbers

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')

print(formatted_output)

- Order IDs
- Current order direction (buy/sell)
- Order quantities
- Order prices
- Order types (market/limit/stop)
- Trading account details
- Brokerage platform API access
- Authentication credentials
- Market conditions
- Trading strategy parameters


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "reverse the direction on all of my orders"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with knowledge in coding and answer without any further explanation or numbers"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')

print(formatted_output)

1. Trading platform or broker API details
2. Authentication credentials (API key, secret, etc.)
3. List of all current open orders
4. Order IDs or unique identifiers
5. Order types (market, limit, stop, etc.)
6. Current order directions (buy/sell)
7. Asset or instrument details (symbol, ticker)
8. Order quantities
9. Price details (if applicable)
10. Time-in-force settings
11. Account details (account ID, margin, etc.)
12. Error handling mechanisms
13. Confirmation and logging mechanisms


In [ ]:
client = OpenAI(api_key='OPENAI_API_KEY')

user_prompt = "reverse the direction on all of my orders"

final_msg_content = f"""
Provide a comprehensive list of all the relevant information needed to execute the following command, without any further explanation or numerical details.

{user_prompt}
"""

messages = [
    {"role": "system", "content":"Pretend you are an expert trader with coding expertise. Provide a complete and detailed response, avoiding any further numerical details"},
    {"role": "user", "content": final_msg_content}
]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  temperature=0,
)

# Process and format the output
output = completion.choices[0].message.content.strip()

# Remove the newlines between bullet points for better formatting
formatted_output = output.replace('\n', '\n')
print(formatted_output)

To execute the command to reverse the direction on all of your orders, you will need the following relevant information:

1. **Account Information:**
   - Account ID
   - Account type (e.g., individual, corporate)

2. **Order Details:**
   - Order IDs
   - Order types (e.g., market, limit, stop)
   - Current order direction (buy or sell)
   - Order quantities
   - Order prices (if applicable)

3. **Asset Information:**
   - Asset symbols or tickers
   - Asset types (e.g., stocks, options, futures, forex)

4. **Brokerage Platform Details:**
   - API access credentials (API key, secret)
   - Endpoint URLs for order management
   - Authentication tokens (if required)

5. **Trading Platform Configuration:**
   - Trading platform software or library being used (e.g., MetaTrader, Interactive Brokers API)
   - Relevant configuration settings (e.g., timeouts, retries)

6. **Market Conditions:**
   - Current market status (open or closed)
   - Market data (e.g., bid/ask prices, volume)

7. **Co